# 1장. Transformer model

- [강좌 링크](https://wikidocs.net/166785)

# 1. 자연어처리 (Natural Language Processing)

## 일반적인 NLP Task 종류와 예시
- 전체 문장 분류(Classifuing whole sentences): 리뷰의 감정을 식별, 스팸 메일 탐지, 문법적으로 올바른지, 두 문장이 논리적으로 관련있는지 판단
- 단일 문장에서 각 단어 분류(Classifying each word in a sentence): 문장의 문법적 구성 요소 또는 명명된 개체 식별(NER, Named Entity Recognition)
- 텍스트 컨텐트 생성(Generating text content): 자동 생성된 텍스트로 프롬프트 완성, 마스킹된 단어로 텍스트 공백 채우기
- 텍스트에서 정답 추출(Extracting an answer from a text): 질문과 맥락이 주어지면 맥락에서 제공된 정보를 기반으로 질문에 대한 답변 추출
- 입력 텍스트에서 새로운 문장 생성(Generating a new sentence from an input text): 텍스트를 다른 언어로 번역하거나 요약

# 2. &#129303;Transformers가 할 수 있는 일들

## 파이프라인(pipeline) 활용하기
&#129303;Transformers 라이브러리의 가장 기본적인 객체는 pipeline() 함수.

특정 모델과 동작에 필요한 전처리 및 후처리 단계를 연결하여 텍스트를 직접 입력하고 이해하기 쉬운 답변을 얻을 수 있음.

In [1]:
from custom_utils import *

from transformers import pipeline

wrapper = CustomObject()

wrapper.classifier = pipeline("sentiment-analysis") # 감정 분석 task와 기본 제공되는 fine-tuned model을 download, caching
wrapper.classifier("I`ve been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9273151755332947}]

또는 여러 개의 문장을 동시에 입력으로 전달할 수도 있다.

In [2]:
print(wrapper.classifier(["I`ve been waiting for a HuggingFace course my whole life.",
            "I hate this so much!!"]))
wrapper.init_wrapper()

[{'label': 'POSITIVE', 'score': 0.9273151755332947}, {'label': 'NEGATIVE', 'score': 0.9993013143539429}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.70GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


파이프라인에 텍스트가 입력되면 3가지 주요 단계가 내부적으로 실행됨.
1. 텍스트가 모델이 이해할 수 있는 형식으로 preprocessing.
2. 전처리 완료된 입력 텍스트가 모델에 전달.
3. 모델이 예측한 결과가 postprocessing되어 사용자가 이해할 수 있는 형태로 반환.

현재 활용 가능한 pipeline들.
- `feature-extraction`(텍스트에 대한 벡터 표현 제공)
- `fill-mask`
- `ner`
- `question-answering`
- `sentiment-analysis`
- `summarization`
- `text-generation`
- `translation`
- `zero-shot-classification`

## Zero-shot Classification
레이블이 지정되지 않은 텍스트를 분류하는 작업

해당 분류에 사용할 레이블을 직접 마음대로 지정할 수 있으므로 사전 훈련된 모델의 레이블 집합에 의존할 필요가 없음.

In [3]:
wrapper.classifier = pipeline("zero-shot-classification")
print(wrapper.classifier(
    "This is a course about the Transformers library",
    candidate_labels = ["education", "politics", "business"]
))
wrapper.init_wrapper()

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course about the Transformers library', 'labels': ['education', 'business', 'politics'], 'scores': [0.8445993065834045, 0.11197393387556076, 0.043426718562841415]}

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.70GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


위와 같이 완전히 다른 새로운 레이블 집합으로 문장 분류를 수행할 때도 새로운 데이터를 이용해 모델을 fine-tuning할 필요가 없기 때문에 zero-shot classification.

예시와 같이 레이블 목록에 대한 확률을 직접 반환할 수도 있음.

## Text Generation

입력으로 특정 프롬프트를 제공하면 모델이 나머지 텍스트를 생성해 프롬프트를 완성.

In [4]:
wrapper.generator = pipeline("text-generation")
print(wrapper.generator("In this course, we will teach you how to"))
wrapper.init_wrapper()

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\jongg\PycharmProjects\HF_Transformers\venv\Lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'In this course, we will teach you how to find the right number of pieces to connect into your guitar.\n\nHow\'s that for my guitar?\n\nLiam & Liam has recently announced the release of a new 5" G2,'}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 0.70GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


## Pipeline에서 허브의 모든 모델을 사용할 수 있다.

task별 default model뿐만이 아니라 hugging face의 모든 모델 중 원하는 모델을 선택해 다운받고 사용할 수 있다.

In [5]:
wrapper.generator = pipeline("text-generation",
                             model = "distilgpt2",
                             device_map = "auto")
print(wrapper.generator(
    "In this course, we will teach you how to",
    max_length = 50,
    num_return_sequences = 3)
)
wrapper.init_wrapper()

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to build and configure a network firewall in order to prevent traffic from being used to attack the server that hosts that use these technologies.\n\n\nPart 2\nOur Tutorial\nPart 3'}, {'generated_text': 'In this course, we will teach you how to do it on the web and in real time to answer questions about the basics of CSS and the real-world practice of CSS\n\n\nSVG\n\nThe Web is open source\nYou can'}, {'generated_text': 'In this course, we will teach you how to use the word “the word \u202c\u202f in conjunction with your own language. We will not need a specific language to understand the word \u202b”. The word ‿†�'}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.93GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


## 추론(Inference) API

모든 모델은 [Hugging Face 웹사이트](https://huggingface.co/)에서 제공되는 Inference API를 사용하여 브라우저를 통해 직접 테스트할 수 있다.

## Fill Mask

주어진 텍스트의 공백을 채우는 것.

모델마다 서로 다른 마스크 토큰을 요구할 수 있으므로 모델 탐색 시 해당 마스크 토큰을 확인해야 함

In [6]:
wrapper.unmasker = pipeline("fill-mask")

"""
:arg top_k 출력할 공백 채우기 결과의 개수를 지정
"""
print(wrapper.unmasker("This course will teach tou all about <mask> models.", top_k = 4))

wrapper.init_wrapper()

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.1958232969045639, 'token': 30412, 'token_str': ' mathematical', 'sequence': 'This course will teach tou all about mathematical models.'}, {'score': 0.04536421224474907, 'token': 38163, 'token_str': ' computational', 'sequence': 'This course will teach tou all about computational models.'}, {'score': 0.029576007276773453, 'token': 27930, 'token_str': ' predictive', 'sequence': 'This course will teach tou all about predictive models.'}, {'score': 0.022237807512283325, 'token': 745, 'token_str': ' building', 'sequence': 'This course will teach tou all about building models.'}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.58GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


## 개체명 인식, NER(Named Entity Recognition)

입력텍스트에서 어느 부분이 사람, 위치 또는 조직과 같은 개체명에 해당하는지 식별

In [7]:
wrapper.ner = pipeline("ner", grouped_entities = True)
print(wrapper.ner("My name is Sylvain and I work at Hugging Face in Brooklyn."))

wrapper.init_wrapper()

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\jongg\PycharmProjects\H

[{'entity_group': 'PER', 'score': 0.9981694, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796019, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9932106, 'word': 'Brooklyn', 'start': 49, 'end': 57}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.58GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


Sylvain이 사람(PER)이고 Hugging Face가 조직(ORG), Brooklyn이 위치(LOC)임을 식별함.

grouped_entities = True 옵션을 통해 동일한 엔티티에 해당하는 토큰 혹은 단어들을 그룹화함.

Sylvain은 S, ##yl, ##va, ##in의 네 토큰으로 나뉘고 postprocess 단계에서 재그룹화하여 Sylvain이 단일 단어로 출력됨

## Question Answering

주어진 context 정보를 사용하여 입력 질문에 응답을 제공

In [8]:
wrapper.question_answerer = pipeline("question-answering")
print(wrapper.question_answerer(
    question = "Whee do I work?",
    context = "My name is Sylvain and I work at Hugging Face in Brooklyn"
))
wrapper.init_wrapper()

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.7483339905738831, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.58GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


## Summarization, 요약

텍스트에 존재하는 중요한 내용을 유지하면서 해당 텍스트를 더 짧은 텍스트로 줄이는 작업

In [9]:
wrapper.summarizer = pipeline("summarization")
print(wrapper.summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
    """
))
wrapper.init_wrapper()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil, electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India, as well as other industrial countries in Europe and Asia, continue to encourage and advance engineering .'}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.58GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


## Translation

Task 이름에 번역할 언어 쌍을 지정하면 default model을 사용할 수 있지만 가장 쉬운 방법은 Model Hub에서 사용하려는 모델을 선택하는 것이다.

In [10]:
# default model

wrapper.translator = pipeline("translation_en_to_fr")
print(wrapper.translator("Had to have high hopes for a living."))
wrapper.init_wrapper()

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\jongg\PycharmProjects\HF_Transformers\venv\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[{'translation_text': 'A dû avoir de grands espoirs pour gagner sa vie.'}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.58GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


In [11]:
# Helsinki-NLP/opus-mt-fr-en

wrapper.translator = pipeline("translation", model = "Helsinki-NLP/opus-mt-fr-en")
print(wrapper.translator("Ce cours est produit par Hugging Face."))
wrapper.init_wrapper()

C:\Users\jongg\PycharmProjects\HF_Transformers\venv\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jongg\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\jongg\PycharmProjects\HF_Transformers\venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'This course is produced by Hugging Face.'}]

GPU NVIDIA GeForce RTX 3060
memory occupied: 1.59GB
Allocated GPU Memory: 0.01GB
Reserved GPU Memory: 0.02GB


# 3. &#129303;Transformers는 어떻게 동작하는가

## Transformer의 짧은 역사

- GPT: 최초의 pretrained transformer model
- BERT: 또 다른 대규모 pretrained model, 고수준의 문장 요약을 제공
- GPT-2: 기존 GPT보다 규모가 더 크고 성능이 향상됨
- DistillBERT: 속도가 60% 더 빠르고 메모리 소비는 40% 줄였지만 여전히 BERT 성능의 97%를 유지하는 증류된(distilled) BERT 버전
- BART 및 T5: Transformer 모델과 동일한 architecture를 사용하는 대규모 pretrained model
- GPT-3: fine-tuning 없이도 다양한 작업을 훌룡하게 수행할 수 있는 GPT-2의 더 큰 버전으로 zero-shot learning이라고 함

**Pretrained Model**
- GPT-like model(**auto-regressive** Transformer model)
- BERT-like model(**auto-encoding** Transformer model)
- BART/T5-like model(**sequence-to-sequence** Transformer model)

## Transformers는 언어모델

위에서 언급한 모든 Transformer 모델은 언어 모델로, 많응 양의 원시 텍스트에 대해 자가 지도 학습되었다.

자가 지도 학습(self-supervised learning)이란 목적 함수가 모델의 입력에서 자동으로 계산되는 학습 유형이므로 사람이 데이터에 레이블을 지정할 필요가 없다.

이런 모델들은 학습된 언어에 대한 통계적인 이해만 가능할 뿐 실제 task에 유용하지 않기 때문에 전이 학습(transfer learning)이라는 프로세스를 거친다.

이 프로세스 동안 모델은 주어진 작업에 대해 감독(supervised) 방식, 즉 사람이 주석으로 추가한 레이블을 사용함으로써 미세 조정(**fine-tuning**)된다.

**Ex).**
n개의 이전 단어를 읽은 문장에서 다음 단어를 예측한다고 할 때 추력할 예측값은 과거 및 현재 입력값에 의존하지만 미래 입력값에는 의존하지 않다.

이것을 **"인과적 언어모델(Causal Language Model)"** 이라고 한다.

## 전이 학습 Transfer Learning

#### 사전 학습 Pretraining
모델을 처음부터 학습하는 작업. 모델의 가중치는 무작위로 초기화되고 사전 지식 없이 학습이 시작되어 매우 많은 양의 데이터가 필요하고 학습 시간도 오래 걸린다.

#### 전이 학습 Transfer Learning
사전 학습된 모델을 확보한 다음 특정 task에 맞는 dataset을 사용하여 추가 학습을 수행

처음부터 직접 학습하지 않고 사전 학습 모델을 사용하는 이유
- pretrained model은 fine-tuning에 사용할 dataset과 유사한 데이터를 바탕으로 이미 학습되어, 사전 학습 과정에서 얻은 지식을 활용할 수 있다.
- 사전 학습된 모델은 이미 많은 데이터에 대해 학습되었기 때문에 fine-tuning 과정에서 훨씬 적은 dataset을 사용하더라도 좋은 결과를 얻을 수 있다.
- 같은 이유로, 좋은 결과를 얻는데 필요한 시간과 자원은 훨씬 적을 수 있다.

## Transformer model architecture

#### 개요
모델은 주로 두 개의 블록으로 구성됨.
- **Encoder**: 입력에 대한 representation 혹은 feature를 도출해 최종 task를 위해 입력에 대한 표현 형태가 최적화되었음을 의미한다.
- **Decoder**: 인코더가 구성한 representation 혹은 feature를 다른 입력과 함께 사용하여 대상 sequence를 생성하는데 이는 모델이 출력 생성에 최적화되어있음을 의미한다.

이들 각각의 블록은 작업의 종류에 따라 개별적으로 사용할 수 있다.
- **인코더 전용 모델**: sentence classification, ner과 같이 입력에 대한 분석 및 이해가 필요한 task에 적합
- **디코더 전용 모델**: text generation 등과 같은 생성 task에 적합
- **인코더-디코더 모델** 혹은 **sequence-to-sequence model**: translation이나 summarization과 같이 입력이 수반되는 생성 task에 적합

#### Attention Layer

Transformer model의 가장 중요한 특징은 attention layer라는 특수 레이어로 구축된다는 것이다. ~~논문 제목부터 "Attention Is All You Need"~~

현재는 **이 레이어가 각 단어의 표현을 처리할 때 문장의 특정 단어들에 특별한 주의를 기울이고 나머지는 거의 무시하도록 모델에 지시할 것**이라는 점만 알면 됨

#### Original Architecture

Transformer 아키텍처는 원래 번역용으로 설계되었다. 인코더가 특정 언어로 된 문장을 입력받고 디코더는 원하는 대상 언어로 표기된 동일한 의미의 문장을 수신한다.

현재 단어에 대한 번역 결과는 문장에서 해당 단어의 앞부분과 뒷부분의 내용에 따라 완전히 달라질 수 있으므로 인코더에서 문장의 모든 단어에 attention을 기울여야 한다.

그러나 디코더는 현재 번역된 단어들에만 attention을 기울인다.~~(번역이 완료되지 않은 단어에 attention을 줘봤자 학습에 방해만 되니까)~~

![](../assets/transformers.png)
왼쪽이 **Encoder**, 오른쪽이 **Decoder**

디코더 블록의 첫 번째 attention layer는 디코더에 대한 모든 과거 입력에 주의를 집중하지만 두 번째 attention layer는 인코더의 출력을 입력으로 받아서 사용하고 있다. 이는 대상 언어가 원본 언어와 다른 단어 순서로 문장을 표현하는 문법 규칙을 가지거나 원본 문장의 뒷부분에 나타난 context가 현재 단어에 대한 최상의 번역을 결정하는데 도움이 될 수 있는 경우 매우 유용하다.

Attention Mask를 통해 인코더/디코더에서 모델이 특정 단어에 attention 적용을 방지할 수 있다. 이는 문장을 batch 처리할 때 특수 패딩 토큰에 적용할 수 있다.

#### Architecture vs Checkpoints

- **Architectures**: 이 용어는 모델의 skeleton을 의미한다. 모델 내의 각 레이어와 연산 등을 정의한다.
- **Checkpoints**: 해당 아키텍처에서 로드될 weight 값들을 의미한다.
- **Model**: Architecture와 Checkpoint보다 덜 명확한 포괄적인 용어이거나 두 가지 모두를 의미할 수도 있다.

# 4. Encoder Models

Transformers 모델의 인코더 모듈만 사용하는 모델.

각 단계에서 attention layer는 초기 입력 문장 전체에 액세스할 수 있으며 bidirectional attention을 수행하는 것이 특징. auto-encoding model이라고도 함.

주어진 초기 문장을 다양한 방법으로 noise를 만들고 다시 원래 문장으로 복구하는 과정을 통해 모델 학습이 진행됨.

#### tasks
- Sentence Classification
- NER
- Word Classification
- Extractive Question Answering

#### 대표적인 모델들
- [ALBERT](https://huggingface.co/docs/transformers/model_doc/albert)
- [BERT](https://huggingface.co/docs/transformers/model_doc/bert)
- [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert)
- [ELECTRA](https://huggingface.co/docs/transformers/model_doc/electra)
- [RoBERTa](https://huggingface.co/docs/transformers/model_doc/roberta)

# 5. Decoder Models

Transformer 모델의 디코더 모듈만 사용하는 모델.

각 단계에서 주어진 단어에 대해 attention layer는 문장에서 현재 처리 단어 앞쪽에 위치한 잔어들에만 액세스할 수 있음.

자동 회귀 모델, auto-regressive model이라고도 함.

문장의 다음 단어 예측을 수행하는 과정을 통해 모델 학습이 진행됨.

#### 대표적인 모델들
- [CTRL](https://huggingface.co/docs/transformers/model_doc/ctrl)
- [GPT](https://huggingface.co/docs/transformers/model_doc/gpt)
- [GPT-2](https://huggingface.co/docs/transformers/model_doc/gpt2)

# 6. Sequence-to-Sequence Models

인코더-디코더 모델은 Transformer 아키텍처의 인코더와 디코더를 모두 사용함.

인코더 또는 디코더 모델의 목적 함수(objectives)를 사용하여 모델 학습이 수행될 수 있지만 약간 더 복잡한 처리 과정이 수반되는 것이 일반적.

#### tasks
- Summarization
- Translation
- Generative Question Answering

#### 대표적인 모델들
- [BART](https://huggingface.co/docs/transformers/model_doc/bart)
- [mBART](https://huggingface.co/docs/transformers/model_doc/mbart)
- [Marian](https://huggingface.co/docs/transformers/model_doc/marian)
- [T5](https://huggingface.co/docs/transformers/model_doc/t5)

# 7. Bias and Limitations

Pretrained model이나 Fine-tuned model을 상용 시스템에서 사용하려는 경우 제약사항(limitation)이 있다.

그 중 가장 크게 이슈가 될 수 있는 사실은 대용량 데이터에 대해 사전 학습을 수행하기 위해 인터넷에 존재하는 나쁜 뎅터까지 수집한다는 것이다.

In [13]:
wrapper.unmasker = pipeline("fill-mask", model = "bert-base-uncased")
wrapper.result = wrapper.unmasker("This man works as a [MASK].")
print([r["token_str"] for r in wrapper.result])

wrapper.result = wrapper.unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in wrapper.result])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


모델은 누락된 단어를 성별과 상관없는 대답은 하나만 제공함.

또한 prostitute(매춘부)는 모델이 "여성" 및 "직업"과 경관되는 상위 5개 단어에 속함.

이는 English Wikipedia와 BookCorpus의 중립적인 데이터를 사용하여 학습된 드문 Transformer 모델 중 하나임에도 불구하고 발생함.